
<h1 id="Supply-Network-Design-2">Supply Network Design 2<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Supply-Network-Design-2">¶</a></h1><h2 id="Objective-and-Prerequisites">Objective and Prerequisites<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Objective-and-Prerequisites">¶</a></h2><p>This model is an extension of the supply network design problem considered in example 19. Given a set of factories, depots, and customers, the goal of this example is to determine which depots to open in order to minimize overall costs. This extenstion allows the possibility to close some depots and open new ones.</p>
<p>This model is example 20 from the fifth edition of Model Building in Mathematical Programming, by H. Paul Williams on pages 275-276 and 332-333.</p>
<p>This example is of beginning difficulty; we assume that you know Python and have some knowledge of the Gurobi Python API and building mathematical optimization models.</p>
<p><strong>Download the Repository</strong> <br/>
You can download the repository containing this and other examples by clicking <a href="https://github.com/Gurobi/modeling-examples/archive/master.zip">here</a>.</p>
<p><strong>Gurobi License</strong> <br/>
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an <a href="https://www.gurobi.com/downloads/request-an-evaluation-license/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-LOG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_supply-network-design_2_COM_EVAL_GITHUB_&amp;utm_term=supply-network-design&amp;utm_content=C_JPM">evaluation license</a> as a <em>commercial user</em>, or download a <a href="https://www.gurobi.com/academia/academic-program-and-licenses/?utm_source=3PW&amp;utm_medium=OT&amp;utm_campaign=WW-MU-LOG-OR-O_LEA-PR_NO-Q3_FY20_WW_JPME_supply-network-design_2_ACADEMIC_EVAL_GITHUB_&amp;utm_term=supply-network-design&amp;utm_content=C_JPM">free license</a> as an <em>academic user</em>.</p>
<hr/>
<h2 id="Problem-Description">Problem Description<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Problem-Description">¶</a></h2><p>In this problem, we have six end customers, each with a known demand for a product.  Customer demand can be satisfied from a set of six depots, or directly from a set of two factories.  Each depot can support a maximum volume of product moving through it, and each factory can produce a maximum amount of product.  There are known costs associated with transporting the product, from a factory to a depot, from a depot to a customer, or from a factory directly to a customer. This extension provides the opportunity to choose which four of the six possible depots to open.  It also provides an option of expanding capacity at one specific depot.</p>
<p>Our supply network has two factories, in Liverpool and Brighton, that produce a product.  Each has a maximum production capacity:</p>
<table>
<thead><tr>
<th>Factory</th>
<th>Supply (tons)</th>
</tr>
</thead>
<tbody>
<tr>
<td>Liverpool</td>
<td>150,000</td>
</tr>
<tr>
<td>Brighton</td>
<td>200,000</td>
</tr>
</tbody>
</table>
<p>The product can be shipped from a factory to a set of six depots.  Each depot has a maximum throughput.  Depots don't produce or consume the product; they simply pass the product through to customers.</p>
<table>
<thead><tr>
<th>Depot</th>
<th>Throughput (tons)</th>
</tr>
</thead>
<tbody>
<tr>
<td>Newcastle</td>
<td>70,000</td>
</tr>
<tr>
<td>Birmingham</td>
<td>50,000</td>
</tr>
<tr>
<td>London</td>
<td>100,000</td>
</tr>
<tr>
<td>Exeter</td>
<td>40,000</td>
</tr>
<tr>
<td>Bristol</td>
<td>30,000</td>
</tr>
<tr>
<td>Northampton</td>
<td>25,000</td>
</tr>
</tbody>
</table>
<p>We can actually only choose four of the six depots to open.  Opening a depot has a cost:</p>
<table>
<thead><tr>
<th>Depot</th>
<th>Cost to open</th>
</tr>
</thead>
<tbody>
<tr>
<td>Newcastle</td>
<td>10,000</td>
</tr>
<tr>
<td>Exeter</td>
<td>5,000</td>
</tr>
<tr>
<td>Bristol</td>
<td>12,000</td>
</tr>
<tr>
<td>Northampton</td>
<td>4,000</td>
</tr>
</tbody>
</table>
<p>(Note that the description in the book talks about the cost of opening Bristol or Northampton, and the savings from closing Newcastle or Exeter, but these are simply different ways of phrasing the same choice).</p>
<p>We also have the option of expanding the capacity at Birmingham by 20,000 tons, for a cost of <span>$</span>3000.</p>
<p>Our network has six customers, each with a given demand.</p>
<table>
<thead><tr>
<th>Customer</th>
<th>Demand (tons)</th>
</tr>
</thead>
<tbody>
<tr>
<td>C1</td>
<td>50,000</td>
</tr>
<tr>
<td>C2</td>
<td>10,000</td>
</tr>
<tr>
<td>C3</td>
<td>40,000</td>
</tr>
<tr>
<td>C4</td>
<td>35,000</td>
</tr>
<tr>
<td>C5</td>
<td>60,000</td>
</tr>
<tr>
<td>C6</td>
<td>20,000</td>
</tr>
</tbody>
</table>
<p>Shipping costs are given in the following table (in dollars per ton).  Columns are source cities and rows are destination cities.  Thus, for example, it costs $1 per ton to ship the product from Liverpool to London.  A '-' in the table indicates that that combination is not possible, so for example it is not possible to ship from the factory in Brighton to the depot in Newcastle.</p>
<table>
<thead><tr>
<th>To</th>
<th>Liverpool</th>
<th>Brighton</th>
<th>Newcastle</th>
<th>Birmingham</th>
<th>London</th>
<th>Exeter</th>
<th>Briston</th>
<th>Northhampton</th>
</tr>
</thead>
<tbody>
<tr>
<td>Depots</td>
</tr>
<tr>
<td>Newcastle</td>
<td>0.5</td>
<td>-</td>
</tr>
<tr>
<td>Birmingham</td>
<td>0.5</td>
<td>0.3</td>
</tr>
<tr>
<td>London</td>
<td>1.0</td>
<td>0.5</td>
</tr>
<tr>
<td>Exeter</td>
<td>0.2</td>
<td>0.2</td>
</tr>
<tr>
<td>Bristol</td>
<td>0.6</td>
<td>0.4</td>
</tr>
<tr>
<td>Northampton</td>
<td>0.4</td>
<td>0.3</td>
</tr>
<tr>
<td>Customers</td>
</tr>
<tr>
<td>C1</td>
<td>1.0</td>
<td>2.0</td>
<td>-</td>
<td>1.0</td>
<td>-</td>
<td>-</td>
<td>1.2</td>
<td>-</td>
</tr>
<tr>
<td>C2</td>
<td>-</td>
<td>-</td>
<td>1.5</td>
<td>0.5</td>
<td>1.5</td>
<td>-</td>
<td>0.6</td>
<td>0.4</td>
</tr>
<tr>
<td>C3</td>
<td>1.5</td>
<td>-</td>
<td>0.5</td>
<td>0.5</td>
<td>2.0</td>
<td>0.2</td>
<td>0.5</td>
<td>-</td>
</tr>
<tr>
<td>C4</td>
<td>2.0</td>
<td>-</td>
<td>1.5</td>
<td>1.0</td>
<td>-</td>
<td>1.5</td>
<td>-</td>
<td>0.5</td>
</tr>
<tr>
<td>C5</td>
<td>-</td>
<td>-</td>
<td>-</td>
<td>0.5</td>
<td>0.5</td>
<td>0.5</td>
<td>0.3</td>
<td>0.6</td>
</tr>
<tr>
<td>C6</td>
<td>1.0</td>
<td>-</td>
<td>1.0</td>
<td>-</td>
<td>1.5</td>
<td>1.5</td>
<td>0.8</td>
<td>0.9</td>
</tr>
</tbody>
</table>
<p>The questions to be answered: (i) Which four depots should be opened? (ii) Should Birmingham be expanded? (iii) Which depots should be used to satisfy customer demand?</p>
<hr/>
<h2 id="Model-Formulation">Model Formulation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Model-Formulation">¶</a></h2><h3 id="Sets-and-Indices">Sets and Indices<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Sets-and-Indices">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Factories&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mtext&gt;Liverpool&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Brighton&lt;/mtext&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-1-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-1" style="width: 19.943em; display: inline-block;"><span style="display: inline-block; position: relative; width: 16.61em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1016.55em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-2"><span class="mi" id="MathJax-Span-3" style="font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-4" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-5" style="font-family: MathJax_Main; padding-left: 0.301em;">Factories</span><span class="mo" id="MathJax-Span-6" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-7" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mtext" id="MathJax-Span-8" style="font-family: MathJax_Main;">Liverpool</span><span class="mo" id="MathJax-Span-9" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-10" style="font-family: MathJax_Main; padding-left: 0.182em;">Brighton</span><span class="mo" id="MathJax-Span-11" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>f</mi><mo>∈</mo><mtext>Factories</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mtext>Liverpool</mtext><mo>,</mo><mtext>Brighton</mtext><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-1" type="math/tex">f \in \text{Factories}=\{\text{Liverpool}, \text{Brighton}\}</script></p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mtext&gt;Newcastle&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Birmingham&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;London&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Exeter&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Bristol&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Northampton&lt;/mtext&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-2-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-12" style="width: 41.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 34.348em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1034.29em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-13"><span class="mi" id="MathJax-Span-14" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-15" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-16" style="font-family: MathJax_Main; padding-left: 0.301em;">Depots</span><span class="mo" id="MathJax-Span-17" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-18" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mtext" id="MathJax-Span-19" style="font-family: MathJax_Main;">Newcastle</span><span class="mo" id="MathJax-Span-20" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-21" style="font-family: MathJax_Main; padding-left: 0.182em;">Birmingham</span><span class="mo" id="MathJax-Span-22" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-23" style="font-family: MathJax_Main; padding-left: 0.182em;">London</span><span class="mo" id="MathJax-Span-24" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-25" style="font-family: MathJax_Main; padding-left: 0.182em;">Exeter</span><span class="mo" id="MathJax-Span-26" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-27" style="font-family: MathJax_Main; padding-left: 0.182em;">Bristol</span><span class="mo" id="MathJax-Span-28" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-29" style="font-family: MathJax_Main; padding-left: 0.182em;">Northampton</span><span class="mo" id="MathJax-Span-30" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi><mo>∈</mo><mtext>Depots</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mtext>Newcastle</mtext><mo>,</mo><mtext>Birmingham</mtext><mo>,</mo><mtext>London</mtext><mo>,</mo><mtext>Exeter</mtext><mo>,</mo><mtext>Bristol</mtext><mo>,</mo><mtext>Northampton</mtext><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-2" type="math/tex">d \in \text{Depots}=\{\text{Newcastle}, \text{Birmingham}, \text{London}, \text{Exeter}, \text{Bristol}, \text{Northampton}\}</script></p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Customers&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mo fence="false" stretchy="false"&gt;{&lt;/mo&gt;&lt;mtext&gt;C1&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;C2&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;C3&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;C4&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;C5&lt;/mtext&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;C6&lt;/mtext&gt;&lt;mo fence="false" stretchy="false"&gt;}&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-3-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-31" style="width: 22.027em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.336em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1018.28em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-32"><span class="mi" id="MathJax-Span-33" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-34" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-35" style="font-family: MathJax_Main; padding-left: 0.301em;">Customers</span><span class="mo" id="MathJax-Span-36" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mo" id="MathJax-Span-37" style="font-family: MathJax_Main; padding-left: 0.301em;">{</span><span class="mtext" id="MathJax-Span-38" style="font-family: MathJax_Main;">C1</span><span class="mo" id="MathJax-Span-39" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-40" style="font-family: MathJax_Main; padding-left: 0.182em;">C2</span><span class="mo" id="MathJax-Span-41" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-42" style="font-family: MathJax_Main; padding-left: 0.182em;">C3</span><span class="mo" id="MathJax-Span-43" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-44" style="font-family: MathJax_Main; padding-left: 0.182em;">C4</span><span class="mo" id="MathJax-Span-45" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-46" style="font-family: MathJax_Main; padding-left: 0.182em;">C5</span><span class="mo" id="MathJax-Span-47" style="font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-48" style="font-family: MathJax_Main; padding-left: 0.182em;">C6</span><span class="mo" id="MathJax-Span-49" style="font-family: MathJax_Main;">}</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi><mo>∈</mo><mtext>Customers</mtext><mo>=</mo><mo fence="false" stretchy="false">{</mo><mtext>C1</mtext><mo>,</mo><mtext>C2</mtext><mo>,</mo><mtext>C3</mtext><mo>,</mo><mtext>C4</mtext><mo>,</mo><mtext>C5</mtext><mo>,</mo><mtext>C6</mtext><mo fence="false" stretchy="false">}</mo></math></span></span><script id="MathJax-Element-3" type="math/tex">c \in \text{Customers}=\{\text{C1}, \text{C2}, \text{C3}, \text{C4}, \text{C5}, \text{C6}\}</script></p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mtext&gt;Factories&lt;/mtext&gt;&lt;mo&gt;&amp;#x222A;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;mo&gt;&amp;#x222A;&lt;/mo&gt;&lt;mtext&gt;Customers&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-4-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-50" style="width: 21.432em; display: inline-block;"><span style="display: inline-block; position: relative; width: 17.86em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1017.8em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-51"><span class="mtext" id="MathJax-Span-52" style="font-family: MathJax_Main;">Cities</span><span class="mo" id="MathJax-Span-53" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mtext" id="MathJax-Span-54" style="font-family: MathJax_Main; padding-left: 0.301em;">Factories</span><span class="mo" id="MathJax-Span-55" style="font-family: MathJax_Main; padding-left: 0.241em;">∪</span><span class="mtext" id="MathJax-Span-56" style="font-family: MathJax_Main; padding-left: 0.241em;">Depots</span><span class="mo" id="MathJax-Span-57" style="font-family: MathJax_Main; padding-left: 0.241em;">∪</span><span class="mtext" id="MathJax-Span-58" style="font-family: MathJax_Main; padding-left: 0.241em;">Customers</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>Cities</mtext><mo>=</mo><mtext>Factories</mtext><mo>∪</mo><mtext>Depots</mtext><mo>∪</mo><mtext>Customers</mtext></math></span></span><script id="MathJax-Element-4" type="math/tex">\text{Cities} = \text{Factories} \cup \text{Depots} \cup \text{Customers}</script></p>
<h3 id="Parameters">Parameters<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Parameters">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;cost&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-5-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-59" style="width: 6.253em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.182em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.18em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-60"><span class="msubsup" id="MathJax-Span-61"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1001.67em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-62" style="font-family: MathJax_Main;">cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.729em;"><span class="texatom" id="MathJax-Span-63"><span class="mrow" id="MathJax-Span-64"><span class="mi" id="MathJax-Span-65" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-66" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-67" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-68" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-69" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-70"><span class="mrow" id="MathJax-Span-71"><span class="mi" id="MathJax-Span-72" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-73" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>cost</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-5" type="math/tex">\text{cost}_{s,t} \in \mathbb{R}^+</script>: Cost of shipping one ton from source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-6-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-74" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-75"><span class="mi" id="MathJax-Span-76" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-6" type="math/tex">s</script> to destination <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-7-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-77" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-78"><span class="mi" id="MathJax-Span-79" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-7" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;supply&lt;/mtext&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-8-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-80" style="width: 7.146em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.955em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.96em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-81"><span class="msubsup" id="MathJax-Span-82"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.86em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-83" style="font-family: MathJax_Main;">supply</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.86em;"><span class="mi" id="MathJax-Span-84" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-85" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-86" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-87"><span class="mrow" id="MathJax-Span-88"><span class="mi" id="MathJax-Span-89" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-90" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.575em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>supply</mtext><mi>f</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-8" type="math/tex">\text{supply}_f \in \mathbb{R}^+</script>: Maximum possible supply from factory <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-9-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-91" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-92"><span class="mi" id="MathJax-Span-93" style="font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.218em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>f</mi></math></span></span><script id="MathJax-Element-9" type="math/tex">f</script> (in tons).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;through&lt;/mtext&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-10-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-94" style="width: 7.801em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.491em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.49em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-95"><span class="msubsup" id="MathJax-Span-96"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-97" style="font-family: MathJax_Main;">through</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.455em;"><span class="mi" id="MathJax-Span-98" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-99" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-100" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-101"><span class="mrow" id="MathJax-Span-102"><span class="mi" id="MathJax-Span-103" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-104" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>through</mtext><mi>d</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-10" type="math/tex">\text{through}_d \in \mathbb{R}^+</script>: Maximum possible flow through depot <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-11-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-105" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-106"><span class="mi" id="MathJax-Span-107" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi></math></span></span><script id="MathJax-Element-11" type="math/tex">d</script> (in tons).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-12-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-108" style="width: 7.741em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.43em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-109"><span class="msubsup" id="MathJax-Span-110"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-111" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="mi" id="MathJax-Span-112" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-113" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-114" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-115"><span class="mrow" id="MathJax-Span-116"><span class="mi" id="MathJax-Span-117" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-118" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>demand</mtext><mi>c</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-12" type="math/tex">\text{demand}_c \in \mathbb{R}^+</script>: Demand for goods at customer <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-13-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-119" style="width: 0.539em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.42em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-120"><span class="mi" id="MathJax-Span-121" style="font-family: MathJax_Math-italic;">c</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>c</mi></math></span></span><script id="MathJax-Element-13" type="math/tex">c</script> (in tons).</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;opencost&lt;/mtext&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;R&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-14-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-122" style="width: 8.217em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.848em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1006.85em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-123"><span class="msubsup" id="MathJax-Span-124"><span style="display: inline-block; position: relative; width: 4.229em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1003.75em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-125" style="font-family: MathJax_Main;">opencost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="mi" id="MathJax-Span-126" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-127" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-128" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-129"><span class="mrow" id="MathJax-Span-130"><span class="mi" id="MathJax-Span-131" style="font-family: MathJax_AMS;">R</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-132" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.432em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>opencost</mtext><mi>d</mi></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">R</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-14" type="math/tex">\text{opencost}_d \in \mathbb{R}^+</script>: Cost of opening depot <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-15-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-133" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-134"><span class="mi" id="MathJax-Span-135" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi></math></span></span><script id="MathJax-Element-15" type="math/tex">d</script> (in dollars).</p>
<h3 id="Decision-Variables">Decision Variables<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Decision-Variables">¶</a></h3><p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;msup&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi mathvariant="double-struck"&gt;N&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;/msup&gt;&lt;/math&gt;' id="MathJax-Element-16-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-136" style="width: 6.372em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.301em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.193em, 1005.3em, 2.682em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-137"><span class="msubsup" id="MathJax-Span-138"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-139" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-140"><span class="mrow" id="MathJax-Span-141"><span class="mi" id="MathJax-Span-142" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-143" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-144" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-145" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="msubsup" id="MathJax-Span-146" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 1.372em; height: 0px;"><span style="position: absolute; clip: rect(3.158em, 1000.72em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="texatom" id="MathJax-Span-147"><span class="mrow" id="MathJax-Span-148"><span class="mi" id="MathJax-Span-149" style="font-family: MathJax_AMS;">N</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -4.402em; left: 0.717em;"><span class="mo" id="MathJax-Span-150" style="font-size: 70.7%; font-family: MathJax_Main;">+</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.425em; border-left: 0px solid; width: 0px; height: 1.504em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub><mo>∈</mo><msup><mrow class="MJX-TeXAtom-ORD"><mi mathvariant="double-struck">N</mi></mrow><mo>+</mo></msup></math></span></span><script id="MathJax-Element-16" type="math/tex">\text{flow}_{s,t} \in \mathbb{N}^+</script>: Quantity of goods (in tons) that is shipped from source <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-17-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-151" style="width: 0.598em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.479em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.61em, 1000.42em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-152"><span class="mi" id="MathJax-Span-153" style="font-family: MathJax_Math-italic;">s</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.718em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>s</mi></math></span></span><script id="MathJax-Element-17" type="math/tex">s</script> to destionation <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-18-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-154" style="width: 0.479em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.36em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1000.3em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-155"><span class="mi" id="MathJax-Span-156" style="font-family: MathJax_Math-italic;">t</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 0.932em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>t</mi></math></span></span><script id="MathJax-Element-18" type="math/tex">t</script>.</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-19-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-157" style="width: 6.967em; display: inline-block;"><span style="display: inline-block; position: relative; width: 5.777em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1005.66em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-158"><span class="msubsup" id="MathJax-Span-159"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-160" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-161"><span class="mrow" id="MathJax-Span-162"><span class="mi" id="MathJax-Span-163" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-164" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-165" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-166" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-167" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-168" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-169" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>open</mtext><mrow class="MJX-TeXAtom-ORD"><mi>d</mi></mrow></msub><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-19" type="math/tex">\text{open}_{d} \in [0,1]</script>: Is depot <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/math&gt;' id="MathJax-Element-20-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-170" style="width: 0.658em; display: inline-block;"><span style="display: inline-block; position: relative; width: 0.539em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.313em, 1000.54em, 2.384em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-171"><span class="mi" id="MathJax-Span-172" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.068em; border-left: 0px solid; width: 0px; height: 1.004em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi>d</mi></math></span></span><script id="MathJax-Element-20" type="math/tex">d</script> open?</p>
<p><span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mtext&gt;expand&lt;/mtext&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mo stretchy="false"&gt;[&lt;/mo&gt;&lt;mn&gt;0&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;mo stretchy="false"&gt;]&lt;/mo&gt;&lt;/math&gt;' id="MathJax-Element-21-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-173" style="width: 7.741em; display: inline-block;"><span style="display: inline-block; position: relative; width: 6.432em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1006.31em, 2.622em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-174"><span class="mtext" id="MathJax-Span-175" style="font-family: MathJax_Main;">expand</span><span class="mo" id="MathJax-Span-176" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mo" id="MathJax-Span-177" style="font-family: MathJax_Main; padding-left: 0.301em;">[</span><span class="mn" id="MathJax-Span-178" style="font-family: MathJax_Main;">0</span><span class="mo" id="MathJax-Span-179" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-180" style="font-family: MathJax_Main; padding-left: 0.182em;">1</span><span class="mo" id="MathJax-Span-181" style="font-family: MathJax_Main;">]</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.354em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mtext>expand</mtext><mo>∈</mo><mo stretchy="false">[</mo><mn>0</mn><mo>,</mo><mn>1</mn><mo stretchy="false">]</mo></math></span></span><script id="MathJax-Element-21" type="math/tex">\text{expand} \in [0,1]</script>: Should Birmingham be expanded?</p>
<h3 id="Objective-Function">Objective Function<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Objective-Function">¶</a></h3><ul>
<li><strong>Cost</strong>: Minimize total shipping costs plus costs of opening depots.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;mtext&gt;Minimize&lt;/mtext&gt;&lt;mspace width="1em" /&gt;&lt;mi&gt;Z&lt;/mi&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mo stretchy="false"&gt;(&lt;/mo&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo stretchy="false"&gt;)&lt;/mo&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;mo&gt;&amp;#x00D7;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;cost&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;opencost&lt;/mtext&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mn&gt;3000&lt;/mn&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mtext&gt;expand&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-22-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-182" style="width: 46.789em; display: inline-block;"><span style="display: inline-block; position: relative; width: 38.991em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1038.99em, 3.812em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-183"><span class="mtext" id="MathJax-Span-184" style="font-family: MathJax_Main;">Minimize</span><span class="mspace" id="MathJax-Span-185" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-186" style="font-family: MathJax_Math-italic;">Z<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-187" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-188" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 5.896em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 2.205em;"><span class="mo" id="MathJax-Span-189" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.277em, 1005.9em, 4.467em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-190"><span class="mrow" id="MathJax-Span-191"><span class="mo" id="MathJax-Span-192" style="font-size: 70.7%; font-family: MathJax_Main;">(</span><span class="mi" id="MathJax-Span-193" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-194" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-195" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-196" style="font-size: 70.7%; font-family: MathJax_Main;">)</span><span class="mo" id="MathJax-Span-197" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-198" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span><span class="mo" id="MathJax-Span-199" style="font-size: 70.7%; font-family: MathJax_Main;">×</span><span class="mtext" id="MathJax-Span-200" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-201" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-202"><span class="msubsup" id="MathJax-Span-203"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1001.67em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-204" style="font-family: MathJax_Main;">cost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.729em;"><span class="texatom" id="MathJax-Span-205"><span class="mrow" id="MathJax-Span-206"><span class="mi" id="MathJax-Span-207" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-208" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-209" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-210" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-211" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-212" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-213"><span class="mrow" id="MathJax-Span-214"><span class="mi" id="MathJax-Span-215" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-216" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-217" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-218" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="munderover" id="MathJax-Span-219" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.777em;"><span class="mo" id="MathJax-Span-220" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.98em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-221"><span class="mrow" id="MathJax-Span-222"><span class="texatom" id="MathJax-Span-223"><span class="mrow" id="MathJax-Span-224"><span class="mi" id="MathJax-Span-225" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span class="mo" id="MathJax-Span-226" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-227" style="font-size: 70.7%; font-family: MathJax_Main;">Depots</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-228" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-229"><span class="msubsup" id="MathJax-Span-230"><span style="display: inline-block; position: relative; width: 4.229em; height: 0px;"><span style="position: absolute; clip: rect(3.217em, 1003.75em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-231" style="font-family: MathJax_Main;">opencost</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.812em;"><span class="mi" id="MathJax-Span-232" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-233" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-234" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-235" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="mi" id="MathJax-Span-236" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-237" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mn" id="MathJax-Span-238" style="font-family: MathJax_Main; padding-left: 0.241em;">3000</span><span class="mo" id="MathJax-Span-239" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mtext" id="MathJax-Span-240" style="font-family: MathJax_Main; padding-left: 0.241em;">expand</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.782em; border-left: 0px solid; width: 0px; height: 3.004em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><mtext>Minimize</mtext><mspace width="1em"></mspace><mi>Z</mi><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mo stretchy="false">(</mo><mi>s</mi><mo>,</mo><mi>t</mi><mo stretchy="false">)</mo><mo>∈</mo><mtext>Cities</mtext><mo>×</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>cost</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub><mo>∗</mo><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>+</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mrow class="MJX-TeXAtom-ORD"><mi>d</mi></mrow><mo>∈</mo><mtext>Depots</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>opencost</mtext><mi>d</mi></msub><mo>∗</mo><msub><mtext>open</mtext><mi>d</mi></msub></mrow><mo>+</mo><mn>3000</mn><mo>∗</mo><mtext>expand</mtext></math></span></span></div><script id="MathJax-Element-22" type="math/tex; mode=display">\begin{equation}
\text{Minimize} \quad Z = \sum_{(s,t) \in \text{Cities} \times \text{Cities}}{\text{cost}_{s,t}*\text{flow}_{s,t}} +
                          \sum_{{d} \in \text{Depots}}{\text{opencost}_d*\text{open}_d} +
                          3000 * \text{expand}
\end{equation}</script><h3 id="Constraints">Constraints<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Constraints">¶</a></h3><ul>
<li><strong>Factory output</strong>: Flow of goods from a factory must respect maximum capacity.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;supply&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;f&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Factories&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-23-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-241" style="width: 21.015em; display: inline-block;"><span style="display: inline-block; position: relative; width: 17.503em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1017.44em, 3.634em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-242"><span class="munderover" id="MathJax-Span-243"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-244" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.5em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-245"><span class="mrow" id="MathJax-Span-246"><span class="mi" id="MathJax-Span-247" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-248" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-249" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-250" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-251"><span class="msubsup" id="MathJax-Span-252"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-253" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-254"><span class="mrow" id="MathJax-Span-255"><span class="mi" id="MathJax-Span-256" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-257" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-258" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-259" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-260" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.336em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1002.86em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-261" style="font-family: MathJax_Main;">supply</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.86em;"><span class="texatom" id="MathJax-Span-262"><span class="mrow" id="MathJax-Span-263"><span class="mi" id="MathJax-Span-264" style="font-size: 70.7%; font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-265" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-266" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-267" style="font-family: MathJax_Math-italic;">f<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.063em;"></span></span><span class="mo" id="MathJax-Span-268" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-269" style="font-family: MathJax_Main; padding-left: 0.301em;">Factories</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.568em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>f</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>supply</mtext><mrow class="MJX-TeXAtom-ORD"><mi>f</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>f</mi><mo>∈</mo><mtext>Factories</mtext></math></span></span></div><script id="MathJax-Element-23" type="math/tex; mode=display">\begin{equation}
\sum_{t \in \text{Cities}}{\text{flow}_{f,t}} \leq \text{supply}_{f} \quad \forall f \in \text{Factories}
\end{equation}</script><ul>
<li><strong>Customer demand</strong>: Flow of goods must meet customer demand.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;demand&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;c&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Customers&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-24-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-270" style="width: 22.324em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.574em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1018.51em, 3.634em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-271"><span class="munderover" id="MathJax-Span-272"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-273" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-274"><span class="mrow" id="MathJax-Span-275"><span class="mi" id="MathJax-Span-276" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-277" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-278" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-279" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-280"><span class="msubsup" id="MathJax-Span-281"><span style="display: inline-block; position: relative; width: 2.741em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-282" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-283"><span class="mrow" id="MathJax-Span-284"><span class="mi" id="MathJax-Span-285" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-286" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-287" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-288" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-289" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.812em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-290" style="font-family: MathJax_Main;">demand</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 3.455em;"><span class="texatom" id="MathJax-Span-291"><span class="mrow" id="MathJax-Span-292"><span class="mi" id="MathJax-Span-293" style="font-size: 70.7%; font-family: MathJax_Math-italic;">c</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-294" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-295" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-296" style="font-family: MathJax_Math-italic;">c</span><span class="mo" id="MathJax-Span-297" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-298" style="font-family: MathJax_Main; padding-left: 0.301em;">Customers</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.568em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>c</mi></mrow></msub></mrow><mo>=</mo><msub><mtext>demand</mtext><mrow class="MJX-TeXAtom-ORD"><mi>c</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>c</mi><mo>∈</mo><mtext>Customers</mtext></math></span></span></div><script id="MathJax-Element-24" type="math/tex; mode=display">\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,c}} = \text{demand}_{c} \quad \forall c \in \text{Customers}
\end{equation}</script><ul>
<li><strong>Depot flow</strong>: Flow into a depot equals flow out of the depot.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;t&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-25-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-299" style="width: 22.443em; display: inline-block;"><span style="display: inline-block; position: relative; width: 18.693em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1018.63em, 3.634em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-300"><span class="munderover" id="MathJax-Span-301"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-302" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-303"><span class="mrow" id="MathJax-Span-304"><span class="mi" id="MathJax-Span-305" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-306" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-307" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-308" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-309"><span class="msubsup" id="MathJax-Span-310"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-311" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-312"><span class="mrow" id="MathJax-Span-313"><span class="mi" id="MathJax-Span-314" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-315" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-316" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-317" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="munderover" id="MathJax-Span-318" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-319" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.5em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-320"><span class="mrow" id="MathJax-Span-321"><span class="mi" id="MathJax-Span-322" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span><span class="mo" id="MathJax-Span-323" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-324" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-325" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-326"><span class="msubsup" id="MathJax-Span-327"><span style="display: inline-block; position: relative; width: 2.682em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-328" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-329"><span class="mrow" id="MathJax-Span-330"><span class="mi" id="MathJax-Span-331" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-332" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-333" style="font-size: 70.7%; font-family: MathJax_Math-italic;">t</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mspace" id="MathJax-Span-334" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-335" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-336" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-337" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-338" style="font-family: MathJax_Main; padding-left: 0.301em;">Depots</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.568em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>d</mi></mrow></msub></mrow><mo>=</mo><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>t</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>d</mi><mo>,</mo><mi>t</mi></mrow></msub></mrow><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>d</mi><mo>∈</mo><mtext>Depots</mtext></math></span></span></div><script id="MathJax-Element-25" type="math/tex; mode=display">\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} = 
\sum_{t \in \text{Cities}}{\text{flow}_{d,t}}
\quad \forall d \in \text{Depots}
\end{equation}</script><ul>
<li><strong>Depot capacity (all but Birmingham)</strong>: Flow into a depot must respect depot capacity, and is only allowed if the depot is open.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;through&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mspace width="1em" /&gt;&lt;mi mathvariant="normal"&gt;&amp;#x2200;&lt;/mi&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;mo&gt;&amp;#x2212;&lt;/mo&gt;&lt;mtext&gt;Birmingham&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-26-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-339" style="width: 32.86em; display: inline-block;"><span style="display: inline-block; position: relative; width: 27.384em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1027.38em, 3.634em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-340"><span class="munderover" id="MathJax-Span-341"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-342" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-343"><span class="mrow" id="MathJax-Span-344"><span class="mi" id="MathJax-Span-345" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-346" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-347" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-348" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-349"><span class="msubsup" id="MathJax-Span-350"><span style="display: inline-block; position: relative; width: 2.801em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-351" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-352"><span class="mrow" id="MathJax-Span-353"><span class="mi" id="MathJax-Span-354" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-355" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mi" id="MathJax-Span-356" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-357" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-358" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 3.872em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-359" style="font-family: MathJax_Main;">through</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.455em;"><span class="texatom" id="MathJax-Span-360"><span class="mrow" id="MathJax-Span-361"><span class="mi" id="MathJax-Span-362" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-363" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="msubsup" id="MathJax-Span-364" style="padding-left: 0.241em;"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-365" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-366"><span class="mrow" id="MathJax-Span-367"><span class="mi" id="MathJax-Span-368" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mspace" id="MathJax-Span-369" style="height: 0em; vertical-align: 0em; width: 1.015em; display: inline-block; overflow: hidden;"></span><span class="mi" id="MathJax-Span-370" style="font-family: MathJax_Main;">∀</span><span class="mi" id="MathJax-Span-371" style="font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-372" style="font-family: MathJax_Main; padding-left: 0.301em;">∈</span><span class="mtext" id="MathJax-Span-373" style="font-family: MathJax_Main; padding-left: 0.301em;">Depots</span><span class="mo" id="MathJax-Span-374" style="font-family: MathJax_Main; padding-left: 0.241em;">−</span><span class="mtext" id="MathJax-Span-375" style="font-family: MathJax_Main; padding-left: 0.241em;">Birmingham</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.568em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mi>d</mi></mrow></msub></mrow><mo>≤</mo><msub><mtext>through</mtext><mrow class="MJX-TeXAtom-ORD"><mi>d</mi></mrow></msub><mo>∗</mo><msub><mtext>open</mtext><mrow class="MJX-TeXAtom-ORD"><mi>d</mi></mrow></msub><mspace width="1em"></mspace><mi mathvariant="normal">∀</mi><mi>d</mi><mo>∈</mo><mtext>Depots</mtext><mo>−</mo><mtext>Birmingham</mtext></math></span></span></div><script id="MathJax-Element-26" type="math/tex; mode=display">\begin{equation}
\sum_{s \in \text{Cities}}{\text{flow}_{s,d}} \leq \text{through}_{d} * \text{open}_{d}
\quad \forall d \in \text{Depots} - \text{Birmingham}
\end{equation}</script><ul>
<li><strong>Depot capacity (Birmingham)</strong>: Flow into Birmingham must respect depot capacity, which may have been expanded.</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Cities&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;msub&gt;&lt;mtext&gt;flow&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;s&lt;/mi&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mtext&gt;Birmingham&lt;/mtext&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;through&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mtext&gt;Birmingham&lt;/mtext&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;+&lt;/mo&gt;&lt;mn&gt;20000&lt;/mn&gt;&lt;mo&gt;&amp;#x2217;&lt;/mo&gt;&lt;mtext&gt;expand&lt;/mtext&gt;&lt;/math&gt;' id="MathJax-Element-27-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-376" style="width: 30.836em; display: inline-block;"><span style="display: inline-block; position: relative; width: 25.658em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1025.66em, 3.634em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-377"><span class="munderover" id="MathJax-Span-378"><span style="display: inline-block; position: relative; width: 2.562em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.539em;"><span class="mo" id="MathJax-Span-379" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.56em, 4.289em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-380"><span class="mrow" id="MathJax-Span-381"><span class="mi" id="MathJax-Span-382" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-383" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-384" style="font-size: 70.7%; font-family: MathJax_Main;">Cities</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="msubsup" id="MathJax-Span-385" style="padding-left: 0.182em;"><span style="display: inline-block; position: relative; width: 6.253em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1001.79em, 4.17em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-386" style="font-family: MathJax_Main;">flow</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.807em; left: 1.789em;"><span class="texatom" id="MathJax-Span-387"><span class="mrow" id="MathJax-Span-388"><span class="mi" id="MathJax-Span-389" style="font-size: 70.7%; font-family: MathJax_Math-italic;">s</span><span class="mo" id="MathJax-Span-390" style="font-size: 70.7%; font-family: MathJax_Main;">,</span><span class="mtext" id="MathJax-Span-391" style="font-size: 70.7%; font-family: MathJax_Main;">Birmingham</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-392" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="msubsup" id="MathJax-Span-393" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 7.384em; height: 0px;"><span style="position: absolute; clip: rect(3.098em, 1003.46em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-394" style="font-family: MathJax_Main;">through</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 3.455em;"><span class="texatom" id="MathJax-Span-395"><span class="mrow" id="MathJax-Span-396"><span class="mtext" id="MathJax-Span-397" style="font-size: 70.7%; font-family: MathJax_Main;">Birmingham</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-398" style="font-family: MathJax_Main; padding-left: 0.241em;">+</span><span class="mn" id="MathJax-Span-399" style="font-family: MathJax_Main; padding-left: 0.241em;">20000</span><span class="mo" id="MathJax-Span-400" style="font-family: MathJax_Main; padding-left: 0.241em;">∗</span><span class="mtext" id="MathJax-Span-401" style="font-family: MathJax_Main; padding-left: 0.241em;">expand</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.568em; border-left: 0px solid; width: 0px; height: 2.789em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>∈</mo><mtext>Cities</mtext></mrow></munder><msub><mtext>flow</mtext><mrow class="MJX-TeXAtom-ORD"><mi>s</mi><mo>,</mo><mtext>Birmingham</mtext></mrow></msub><mo>≤</mo><msub><mtext>through</mtext><mrow class="MJX-TeXAtom-ORD"><mtext>Birmingham</mtext></mrow></msub><mo>+</mo><mn>20000</mn><mo>∗</mo><mtext>expand</mtext></math></span></span></div><script id="MathJax-Element-27" type="math/tex; mode=display">\begin{equation}
\sum_{s \in \text{Cities}} \text{flow}_{s,\text{Birmingham}} \leq \text{through}_{\text{Birmingham}} + 20000 * \text{expand}
\end{equation}</script><ul>
<li><strong>Open depots</strong>: At most 4 open depots (no choice for Birmingham or London).</li>
</ul>
<span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;munder&gt;&lt;mo&gt;&amp;#x2211;&lt;/mo&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;mo&gt;&amp;#x2208;&lt;/mo&gt;&lt;mtext&gt;Depots&lt;/mtext&gt;&lt;/mrow&gt;&lt;/munder&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mi&gt;d&lt;/mi&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;/mrow&gt;&lt;mo&gt;&amp;#x2264;&lt;/mo&gt;&lt;mn&gt;4&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-28-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-402" style="width: 9.11em; display: inline-block;"><span style="display: inline-block; position: relative; width: 7.562em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.074em, 1007.56em, 3.753em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-403"><span class="munderover" id="MathJax-Span-404"><span style="display: inline-block; position: relative; width: 2.979em; height: 0px;"><span style="position: absolute; clip: rect(2.86em, 1001.37em, 4.646em, -999.997em); top: -3.985em; left: 0.777em;"><span class="mo" id="MathJax-Span-405" style="font-family: MathJax_Size2; vertical-align: 0em;">∑</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; clip: rect(3.336em, 1002.98em, 4.408em, -999.997em); top: -2.854em; left: 0em;"><span class="texatom" id="MathJax-Span-406"><span class="mrow" id="MathJax-Span-407"><span class="mi" id="MathJax-Span-408" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span><span class="mo" id="MathJax-Span-409" style="font-size: 70.7%; font-family: MathJax_Main;">∈</span><span class="mtext" id="MathJax-Span-410" style="font-size: 70.7%; font-family: MathJax_Main;">Depots</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="texatom" id="MathJax-Span-411" style="padding-left: 0.182em;"><span class="mrow" id="MathJax-Span-412"><span class="msubsup" id="MathJax-Span-413"><span style="display: inline-block; position: relative; width: 2.503em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-414" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-415"><span class="mrow" id="MathJax-Span-416"><span class="mi" id="MathJax-Span-417" style="font-size: 70.7%; font-family: MathJax_Math-italic;">d<span style="display: inline-block; overflow: hidden; height: 1px; width: 0.003em;"></span></span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span></span></span><span class="mo" id="MathJax-Span-418" style="font-family: MathJax_Main; padding-left: 0.301em;">≤</span><span class="mn" id="MathJax-Span-419" style="font-family: MathJax_Main; padding-left: 0.301em;">4</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -1.711em; border-left: 0px solid; width: 0px; height: 2.932em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><munder><mo>∑</mo><mrow class="MJX-TeXAtom-ORD"><mi>d</mi><mo>∈</mo><mtext>Depots</mtext></mrow></munder><mrow class="MJX-TeXAtom-ORD"><msub><mtext>open</mtext><mrow class="MJX-TeXAtom-ORD"><mi>d</mi></mrow></msub></mrow><mo>≤</mo><mn>4</mn></math></span></span></div><script id="MathJax-Element-28" type="math/tex; mode=display">\begin{equation}
\sum_{d \in \text{Depots}}{\text{open}_{d}} \leq 4
\end{equation}</script><span class="MathJax_Preview" style="color: inherit; display: none;"></span><div class="MathJax_Display" style="text-align: center;"><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML" display="block"&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mtext&gt;Birmingham&lt;/mtext&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;msub&gt;&lt;mtext&gt;open&lt;/mtext&gt;&lt;mrow class="MJX-TeXAtom-ORD"&gt;&lt;mtext&gt;London&lt;/mtext&gt;&lt;/mrow&gt;&lt;/msub&gt;&lt;mo&gt;=&lt;/mo&gt;&lt;mn&gt;1&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-29-Frame" role="presentation" style="text-align: center; position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-420" style="width: 16.432em; display: inline-block;"><span style="display: inline-block; position: relative; width: 13.693em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.372em, 1013.63em, 2.741em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-421"><span class="msubsup" id="MathJax-Span-422"><span style="display: inline-block; position: relative; width: 5.955em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-423" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-424"><span class="mrow" id="MathJax-Span-425"><span class="mtext" id="MathJax-Span-426" style="font-size: 70.7%; font-family: MathJax_Main;">Birmingham</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-427" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="msubsup" id="MathJax-Span-428" style="padding-left: 0.301em;"><span style="display: inline-block; position: relative; width: 4.467em; height: 0px;"><span style="position: absolute; clip: rect(3.336em, 1002.03em, 4.348em, -999.997em); top: -3.985em; left: 0em;"><span class="mtext" id="MathJax-Span-429" style="font-family: MathJax_Main;">open</span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span><span style="position: absolute; top: -3.747em; left: 2.086em;"><span class="texatom" id="MathJax-Span-430"><span class="mrow" id="MathJax-Span-431"><span class="mtext" id="MathJax-Span-432" style="font-size: 70.7%; font-family: MathJax_Main;">London</span></span></span><span style="display: inline-block; width: 0px; height: 3.991em;"></span></span></span></span><span class="mo" id="MathJax-Span-433" style="font-family: MathJax_Main; padding-left: 0.301em;">=</span><span class="mn" id="MathJax-Span-434" style="font-family: MathJax_Main; padding-left: 0.301em;">1</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.496em; border-left: 0px solid; width: 0px; height: 1.361em;"></span></span></nobr><span class="MJX_Assistive_MathML MJX_Assistive_MathML_Block" role="presentation"><math display="block" xmlns="http://www.w3.org/1998/Math/MathML"><msub><mtext>open</mtext><mrow class="MJX-TeXAtom-ORD"><mtext>Birmingham</mtext></mrow></msub><mo>=</mo><msub><mtext>open</mtext><mrow class="MJX-TeXAtom-ORD"><mtext>London</mtext></mrow></msub><mo>=</mo><mn>1</mn></math></span></span></div><script id="MathJax-Element-29" type="math/tex; mode=display">\begin{equation}
\text{open}_{\text{Birmingham}} = \text{open}_{\text{London}} = 1
\end{equation}</script><hr/>
<h2 id="Python-Implementation">Python Implementation<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Python-Implementation">¶</a></h2><p>We import the Gurobi Python Module and other Python libraries.</p>


In [ ]:

import numpy as np
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0




<h2 id="Input-Data">Input Data<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Input-Data">¶</a></h2><p>We define all the input data for the model.</p>


In [ ]:

# Create dictionaries to capture factory supply limits, depot throughput limits, cost of opening depots, and customer demand.

supply = dict({'Liverpool': 150000,
               'Brighton': 200000})

through = dict({'Newcastle': 70000,
                'Birmingham': 50000,
                'London': 100000,
                'Exeter': 40000,
                'Bristol': 30000,
                'Northampton': 25000})

opencost = dict({'Newcastle': 10000,
                 'Birmingham': 0,
                 'London': 0,
                 'Exeter': 5000,
                 'Bristol': 12000,
                 'Northampton': 4000})

demand = dict({'C1': 50000,
               'C2': 10000,
               'C3': 40000,
               'C4': 35000,
               'C5': 60000,
               'C6': 20000})

# Create a dictionary to capture shipping costs.

arcs, cost = gp.multidict({
    ('Liverpool', 'Newcastle'): 0.5,
    ('Liverpool', 'Birmingham'): 0.5,
    ('Liverpool', 'London'): 1.0,
    ('Liverpool', 'Exeter'): 0.2,
    ('Liverpool', 'Bristol'): 0.6,
    ('Liverpool', 'Northampton'): 0.4,
    ('Liverpool', 'C1'): 1.0,
    ('Liverpool', 'C3'): 1.5,
    ('Liverpool', 'C4'): 2.0,
    ('Liverpool', 'C6'): 1.0,
    ('Brighton', 'Birmingham'): 0.3,
    ('Brighton', 'London'): 0.5,
    ('Brighton', 'Exeter'): 0.2,
    ('Brighton', 'Bristol'): 0.4,
    ('Brighton', 'Northampton'): 0.3,
    ('Brighton', 'C1'): 2.0,
    ('Newcastle', 'C2'): 1.5,
    ('Newcastle', 'C3'): 0.5,
    ('Newcastle', 'C5'): 1.5,
    ('Newcastle', 'C6'): 1.0,
    ('Birmingham', 'C1'): 1.0,
    ('Birmingham', 'C2'): 0.5,
    ('Birmingham', 'C3'): 0.5,
    ('Birmingham', 'C4'): 1.0,
    ('Birmingham', 'C5'): 0.5,
    ('London', 'C2'): 1.5,
    ('London', 'C3'): 2.0,
    ('London', 'C5'): 0.5,
    ('London', 'C6'): 1.5,
    ('Exeter', 'C3'): 0.2,
    ('Exeter', 'C4'): 1.5,
    ('Exeter', 'C5'): 0.5,
    ('Exeter', 'C6'): 1.5,
    ('Bristol', 'C1'): 1.2,
    ('Bristol', 'C2'): 0.6,
    ('Bristol', 'C3'): 0.5,
    ('Bristol', 'C5'): 0.3,
    ('Bristol', 'C6'): 0.8,
    ('Northampton', 'C2'): 0.4,
    ('Northampton', 'C4'): 0.5,
    ('Northampton', 'C5'): 0.6,
    ('Northampton', 'C6'): 0.9
})




<h2 id="Model-Deployment">Model Deployment<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Model-Deployment">¶</a></h2><p>We create a model and the variables. The 'flow' variables simply capture the amount of product that flows along each allowed path between a source and destination.  The 'open' variable capture decisions about which depots to open.  The 'expand' variable captures the choice of whether to expand Birmingham.  Objective coefficients are provided here, so we don't need to provide an optimization objective later.</p>


In [ ]:

model = gp.Model('SupplyNetworkDesign2')

depots = through.keys()
flow = model.addVars(arcs, obj=cost, name="flow")
open = model.addVars(depots, obj=opencost, vtype=GRB.BINARY, name="open")
expand = model.addVar(obj=3000, vtype=GRB.BINARY, name="expand")

open['Birmingham'].lb = 1
open['London'].lb = 1
model.objcon = -(opencost['Newcastle'] + opencost['Exeter']) # Phrased as 'savings from closing'




<p>Our first constraints require the total flow along arcs leaving a factory to be at most as large as the supply capacity of that factory.</p>


In [ ]:

# Production capacity limits

factories = supply.keys()
factory_flow = model.addConstrs((gp.quicksum(flow.select(factory, '*')) <= supply[factory]
                                 for factory in factories), name="factory")




<p>Our next constraints require the total flow along arcs entering a customer to be equal to the demand from that customer.</p>


In [ ]:

# Customer demand

customers = demand.keys()
customer_flow = model.addConstrs((gp.quicksum(flow.select('*', customer)) == demand[customer]
                                  for customer in customers), name="customer")




<p>Our final constraints relate to depots.  The first constraints require that the total amount of product entering the depot must equal the total amount leaving.</p>


In [ ]:

# Depot flow conservation

depot_flow = model.addConstrs((gp.quicksum(flow.select(depot, '*')) == gp.quicksum(flow.select('*', depot))
                               for depot in depots), name="depot")




<p>The second set limits the product passing through the depot to be at most equal the throughput of that deport, or 0 if the depot isn't open.</p>


In [ ]:

# Depot throughput

all_but_birmingham = list(set(depots) - set(['Birmingham']))

depot_capacity = model.addConstrs((gp.quicksum(flow.select(depot, '*')) <= through[depot]*open[depot]
                                   for depot in all_but_birmingham), name="depot_capacity")




<p>The capacity constraint for Birmingham is different.  The depot is always open, but we have the option of expanding its capacity.</p>


In [ ]:

birmingham_capacity = model.addConstr(gp.quicksum(flow.select('*', 'Birmingham')) <= through['Birmingham'] +
                                      20000*expand, name="birmingham_capacity")




<p>Finally, there's a limit of at most 4 open depots</p>


In [ ]:

# Depot count

depot_count = model.addConstr(open.sum() <= 4)




<p>We now optimize the model</p>


In [ ]:

model.optimize()




<hr/>
<h2 id="Analysis">Analysis<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#Analysis">¶</a></h2><p>The product demand from all of our customers can be satisfied for a total cost of <span class="MathJax_Preview" style="color: inherit; display: none;"></span><span class="MathJax" data-mathml='&lt;math xmlns="http://www.w3.org/1998/Math/MathML"&gt;&lt;mi mathvariant="normal"&gt;&amp;#x0024;&lt;/mi&gt;&lt;mn&gt;174&lt;/mn&gt;&lt;mo&gt;,&lt;/mo&gt;&lt;mn&gt;000&lt;/mn&gt;&lt;/math&gt;' id="MathJax-Element-30-Frame" role="presentation" style="position: relative;" tabindex="0"><nobr aria-hidden="true"><span class="math" id="MathJax-Span-435" style="width: 4.824em; display: inline-block;"><span style="display: inline-block; position: relative; width: 3.991em; height: 0px; font-size: 120%;"><span style="position: absolute; clip: rect(1.253em, 1003.93em, 2.562em, -999.997em); top: -2.199em; left: 0em;"><span class="mrow" id="MathJax-Span-436"><span class="mi" id="MathJax-Span-437" style="font-family: MathJax_Main;">$</span><span class="mn" id="MathJax-Span-438" style="font-family: MathJax_Main;">174</span><span class="mo" id="MathJax-Span-439" style="font-family: MathJax_Main;">,</span><span class="mn" id="MathJax-Span-440" style="font-family: MathJax_Main; padding-left: 0.182em;">000</span></span><span style="display: inline-block; width: 0px; height: 2.205em;"></span></span></span><span style="display: inline-block; overflow: hidden; vertical-align: -0.282em; border-left: 0px solid; width: 0px; height: 1.289em;"></span></span></nobr><span class="MJX_Assistive_MathML" role="presentation"><math xmlns="http://www.w3.org/1998/Math/MathML"><mi mathvariant="normal">$</mi><mn>174</mn><mo>,</mo><mn>000</mn></math></span></span><script id="MathJax-Element-30" type="math/tex">\$174,000</script> by opening a depot in Northampton, closing the depot in Newcastle, and expanding the depot in Birmingham:</p>


In [ ]:

print('List of open depots:', [d for d in depots if open[d].x > 0.5])
if expand.x > 0.5:
    print('Expand Birmingham')



In [ ]:

product_flow = pd.DataFrame(columns=["From", "To", "Flow"])
for arc in arcs:
    if flow[arc].x > 1e-6:
        product_flow = product_flow.append({"From": arc[0], "To": arc[1], "Flow": flow[arc].x}, ignore_index=True)  
product_flow.index=[''] * len(product_flow)
product_flow




<hr/>
<h2 id="References">References<a class="anchor-link" href="https://gurobi.github.io/modeling-examples/supply_network_design_1_2/supply_network_design_2.html#References">¶</a></h2><p>H. Paul Williams, Model Building in Mathematical Programming, fifth edition.</p>
<p>Copyright © 2020 Gurobi Optimization, LLC</p>
